In [2]:
%matplotlib inline

In [20]:
from scipy import io as sio
import matplotlib.pyplot as plt
import numpy as np
# Package imports
import matplotlib.pyplot as plt
import numpy as np
from random import randrange
import matplotlib

matplotlib.rcParams['figure.figsize'] = (10.0, 8.0)

In [5]:
def predict(row, weights):
    activation = weights[0]
    for i in range(len(row)-1):
        activation += weights[i + 1] * row[i]
    return 1.0 if activation >= 0.0 else 0.0

In [6]:
def train_weights(train, l_rate, n_epoch):
    weights = [0.0 for i in range(len(train[0]))]
    for epoch in range(n_epoch):
        sum_error = 0.0
        for row in train:
            prediction = predict(row, weights)
            error = row[-1] - prediction
            sum_error += error**2
            weights[0] = weights[0] + l_rate * error
            for i in range(len(row)-1):
                weights[i + 1] = weights[i + 1] + l_rate * error * row[i]
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
    return weights

In [18]:
def perceptron(train, test, l_rate, n_epoch):
    predictions = list()
    weights = train_weights(train, l_rate, n_epoch)
    for row in test:
        prediction = predict(row, weights)
        predictions.append(prediction)
    return(predictions)

In [70]:
l_rate = 0.1
n_epoch = 10
n_folds = 2

In [66]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split
 
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0
 
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
#     folds = dataset
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

In [72]:
X = np.array(sio.loadmat('data/concentric.mat')['X'])
X.shape

(2, 152)

In [73]:
scores = evaluate_algorithm(X, perceptron, n_folds, l_rate, n_epoch)

>epoch=0, lrate=0.100, error=0.637
>epoch=1, lrate=0.100, error=0.041
>epoch=2, lrate=0.100, error=0.041
>epoch=3, lrate=0.100, error=0.041
>epoch=4, lrate=0.100, error=0.041
>epoch=5, lrate=0.100, error=0.637
>epoch=6, lrate=0.100, error=0.041
>epoch=7, lrate=0.100, error=0.041
>epoch=8, lrate=0.100, error=0.041
>epoch=9, lrate=0.100, error=0.041


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [74]:
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Mean Accuracy: 1.333%
